<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PUT-FUNCTION-INTO-.py-FILE" data-toc-modified-id="PUT-FUNCTION-INTO-.py-FILE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PUT FUNCTION INTO <code>.py</code> FILE</a></span></li><li><span><a href="#Additional-Database" data-toc-modified-id="Additional-Database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Additional Database</a></span></li><li><span><a href="#Link1" data-toc-modified-id="Link1-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Link1</a></span></li><li><span><a href="#Link2" data-toc-modified-id="Link2-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Link2</a></span></li><li><span><a href="#Link3" data-toc-modified-id="Link3-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Link3</a></span></li><li><span><a href="#Link4" data-toc-modified-id="Link4-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Link4</a></span></li><li><span><a href="#Link5" data-toc-modified-id="Link5-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Link5</a></span></li></ul></div>

Going to practice ripping text from PDFs enable to put them into a database.

In [126]:
import PyPDF2
import re
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)


## PUT FUNCTION INTO `.py` FILE

In [3]:
pageObj = pdfReader.getPage(0)
al = pageObj.extractText()

So there is a head to this document and we need to remove it. Since it will not be standard slice (every state has a different length of a name), we will have to see if there are patterns between the documents that we can exploit.

In [4]:
al

'U.S. Military Fatal Casualties of the Vietnam War for \nHome-State-of-Record: Alabama\nName\nService\nRank / \nRate\n   Birthdate \n(YYYYMMDD)\nIncident or \nDeath Date \n(YYYYMMDD)\nRemains \nRecovered\nHome of Record:\nCity                  County\nABRAMS TIMOTHY C JR\nARMY\nPFC\n19461104\nTUSCALOOSA\nTUSCALOOSA\n19670103\nY\nABSTON JAMES ESTUS JR\nARMY\nPFC\n19441102\nCOTTONDALE\nTUSCALOOSA\n19670227\nY\nACTON MARION \nFRANKLIN\nARMY\nPFC\n19470901\nHUNTSVILLE\nMADISON\n19660411\nY\nADAMS JAMES CONRAD\nARMY\nMAJ\n19381025\nPHENIX CITY\nRUSSELL\n19700501\nY\nADAMS SPENCER\nARMY\nSSG\n19340506\nMOBILE\nMOBILE\n19680215\nY\nADAMS WALTER LEE\nARMY\nPFC\n19461227\nMCINTOSH\nWASHINGTON\n19670610\nY\nADAMSON LARRY ONEAL\nARMY\nSP4\n19460910\nFAIRFAX\nCHAMBERS\n19691003\nY\nADDISON ONEAL\nARMY\nSP5\n19270506\nWEST ENSLEY\nJEFFERSON\n19660504\nY\nADKISON CARL ELMUS\nARMY\nSP4\n19490531\nGADSDEN\nETOWAH\n19690812\nY\nAKINS ADRIAN ALAN\nARMY\nSP5\n19490404\nCORDOVA\nWALKER\n19691025\nY\nALEXA

In [5]:
# pdf file object
# you can find find the pdf file with complete code in below
pdfFileObj = open('state_pdfs/ak-alpha.pdf', 'rb')
# pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
# number of pages in pdf
pages = pdfReader.numPages
pages

5

In [6]:
pageObj = pdfReader.getPage(0)
ak = pageObj.extractText()

In [7]:
ak

'U.S. Military Fatal Casualties of the Vietnam War for \nHome-State-of-Record: Alaska\nName\nService\nRank / \nRate\n   Birthdate \n(YYYYMMDD)\nIncident or \nDeath Date \n(YYYYMMDD)\nRemains \nRecovered\nHome of Record:\nCity                  County\nANDERSON THOMAS \nEDWARD\nMARINE CORPS\nCPL\n19390727\nSPENARD\nANCHORAGE\n19621006\nN\nBANTA MICHAEL DEAN\nARMY\nPFC\n19491005\nCORDOVA\nVALDEZ-\nCORDOVA\n19701002\nY\nBARR EDWARD \nNASUESAK\nNAVY\nSN\n19491022\nBREVIG \nMISSION\nNOME\n19690503\nY\nBARR THOMAS M\nARMY\nSP5\n19450701\nANCHORAGE\nANCHORAGE\n19690512\nY\nBAUER RICHARD GENE\nARMY\nSP5\n19500619\nANCHORAGE\nANCHORAGE\n19691104\nY\nBETTS LARRY LE ROY\nARMY\n2LT\n19461011\nEAGLE RIVER\nANCHORAGE\n19691121\nY\nBROWN CHARLES \nEDWARD\nARMY\nPFC\n19421003\nTHORNE BAY\nPRINCE OF \nWALES-OUTER \nKETCHIKAN\n19671206\nY\nBROWN DAVID DEE JR\nMARINE CORPS\nPFC\n19491010\nWRANGELL\nWRANGELL-\nPETERSBURG\n19680416\nY\nBULLOCK GARY EDWIN\nARMY\nSP4\n19420918\nANCHORAGE\nANCHORAGE\n19670131\

___
For both of these records, all the data exists between the words "County" and "Source". I will grab everything in between the two words.
___

In [8]:
m = re.search(r"(?<=County\n).*(?=Source)", ak, flags=re.DOTALL) #re.DOTALL - makes the '.' special character match any character at all, including a newline
m.group(0)

'ANDERSON THOMAS \nEDWARD\nMARINE CORPS\nCPL\n19390727\nSPENARD\nANCHORAGE\n19621006\nN\nBANTA MICHAEL DEAN\nARMY\nPFC\n19491005\nCORDOVA\nVALDEZ-\nCORDOVA\n19701002\nY\nBARR EDWARD \nNASUESAK\nNAVY\nSN\n19491022\nBREVIG \nMISSION\nNOME\n19690503\nY\nBARR THOMAS M\nARMY\nSP5\n19450701\nANCHORAGE\nANCHORAGE\n19690512\nY\nBAUER RICHARD GENE\nARMY\nSP5\n19500619\nANCHORAGE\nANCHORAGE\n19691104\nY\nBETTS LARRY LE ROY\nARMY\n2LT\n19461011\nEAGLE RIVER\nANCHORAGE\n19691121\nY\nBROWN CHARLES \nEDWARD\nARMY\nPFC\n19421003\nTHORNE BAY\nPRINCE OF \nWALES-OUTER \nKETCHIKAN\n19671206\nY\nBROWN DAVID DEE JR\nMARINE CORPS\nPFC\n19491010\nWRANGELL\nWRANGELL-\nPETERSBURG\n19680416\nY\nBULLOCK GARY EDWIN\nARMY\nSP4\n19420918\nANCHORAGE\nANCHORAGE\n19670131\nY\nCHILDERS WILLIAM \nSTEVEN\nARMY\nWO1\n19470523\nFAIRBANKS\nFAIRBANKS \nNORTH STAR\n19690103\nY\nCHMIEL DONALD GEORGE\nARMY\nSSG\n19440801\nCHUGIAK\nANCHORAGE\n19700825\nY\nCOOK CLINTON ARTHUR\nARMY\nSGT\n19481013\nHYDABURG\nPRINCE OF \nWALES-OUTE

In [9]:
m = re.search(r"(?<=County\n).*(?=Source)", al, flags=re.DOTALL) #re.DOTALL - makes the '.' special character match any character at all, including a newline
m.group(0)

'ABRAMS TIMOTHY C JR\nARMY\nPFC\n19461104\nTUSCALOOSA\nTUSCALOOSA\n19670103\nY\nABSTON JAMES ESTUS JR\nARMY\nPFC\n19441102\nCOTTONDALE\nTUSCALOOSA\n19670227\nY\nACTON MARION \nFRANKLIN\nARMY\nPFC\n19470901\nHUNTSVILLE\nMADISON\n19660411\nY\nADAMS JAMES CONRAD\nARMY\nMAJ\n19381025\nPHENIX CITY\nRUSSELL\n19700501\nY\nADAMS SPENCER\nARMY\nSSG\n19340506\nMOBILE\nMOBILE\n19680215\nY\nADAMS WALTER LEE\nARMY\nPFC\n19461227\nMCINTOSH\nWASHINGTON\n19670610\nY\nADAMSON LARRY ONEAL\nARMY\nSP4\n19460910\nFAIRFAX\nCHAMBERS\n19691003\nY\nADDISON ONEAL\nARMY\nSP5\n19270506\nWEST ENSLEY\nJEFFERSON\n19660504\nY\nADKISON CARL ELMUS\nARMY\nSP4\n19490531\nGADSDEN\nETOWAH\n19690812\nY\nAKINS ADRIAN ALAN\nARMY\nSP5\n19490404\nCORDOVA\nWALKER\n19691025\nY\nALEXANDER BOBBY RAY\nMARINE CORPS\nCPL\n19470213\nDECATUR\nMORGAN\n19671116\nY\nALEXANDER DAVID J JR\nARMY\nPFC\n19480228\nANNISTON\nCALHOUN\n19680718\nY\nALLEN GRANVILLE JOEL \nJR\nNAVY\nPO3\n19471102\nBIRMINGHAM\nJEFFERSON\n19680523\nY\nALLEN JOHNNY JR\n

In [10]:
m = re.search(r"(?<=County\n).*(?=Source)", al, flags=re.DOTALL) #re.DOTALL - makes the '.' special character match any character at all, including a newline
trial = m.group(0)
trial

'ABRAMS TIMOTHY C JR\nARMY\nPFC\n19461104\nTUSCALOOSA\nTUSCALOOSA\n19670103\nY\nABSTON JAMES ESTUS JR\nARMY\nPFC\n19441102\nCOTTONDALE\nTUSCALOOSA\n19670227\nY\nACTON MARION \nFRANKLIN\nARMY\nPFC\n19470901\nHUNTSVILLE\nMADISON\n19660411\nY\nADAMS JAMES CONRAD\nARMY\nMAJ\n19381025\nPHENIX CITY\nRUSSELL\n19700501\nY\nADAMS SPENCER\nARMY\nSSG\n19340506\nMOBILE\nMOBILE\n19680215\nY\nADAMS WALTER LEE\nARMY\nPFC\n19461227\nMCINTOSH\nWASHINGTON\n19670610\nY\nADAMSON LARRY ONEAL\nARMY\nSP4\n19460910\nFAIRFAX\nCHAMBERS\n19691003\nY\nADDISON ONEAL\nARMY\nSP5\n19270506\nWEST ENSLEY\nJEFFERSON\n19660504\nY\nADKISON CARL ELMUS\nARMY\nSP4\n19490531\nGADSDEN\nETOWAH\n19690812\nY\nAKINS ADRIAN ALAN\nARMY\nSP5\n19490404\nCORDOVA\nWALKER\n19691025\nY\nALEXANDER BOBBY RAY\nMARINE CORPS\nCPL\n19470213\nDECATUR\nMORGAN\n19671116\nY\nALEXANDER DAVID J JR\nARMY\nPFC\n19480228\nANNISTON\nCALHOUN\n19680718\nY\nALLEN GRANVILLE JOEL \nJR\nNAVY\nPO3\n19471102\nBIRMINGHAM\nJEFFERSON\n19680523\nY\nALLEN JOHNNY JR\n

In [11]:
state_list = trial.splitlines()  #breaks on the newline
print(len(state_list))
state_list[:10]

130


['ABRAMS TIMOTHY C JR',
 'ARMY',
 'PFC',
 '19461104',
 'TUSCALOOSA',
 'TUSCALOOSA',
 '19670103',
 'Y',
 'ABSTON JAMES ESTUS JR',
 'ARMY']

Great. Now that I can get the data into lists with an indiviual entries.

OK, now lets make a function that goes through each PDF to scrape the data. I will use the function above to put the data into the proper column. Then I will embed that into a loop that will go through all the PDFs in the folder.

In [12]:
def scrape_pdf(pdf_file): #Accepts in a PDF and returns the text of the PDF broken at newlines.
    
    pdfFileObj = open(pdf_file, 'rb')               #Opens file
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)    #Reads the file
    pages = pdfReader.numPages                      #Gets the number of pages to cycle through
    
    counter = 0
    data_list = []
    
    while counter < pages:
        
        pageObj = pdfReader.getPage(counter)
        text = pageObj.extractText()
        
        m = re.search(r"(?<=County\n).*(?=Source)", text, flags=re.DOTALL) #re.DOTALL - makes the '.' special character match any character at all, including a newline
        data = m.group(0)
        data_list.extend(data.splitlines())
        
        counter += 1
        
    data_list.pop() #The last thing at the end of the PDF, on the last page, is some text. Removing it will make it easier to deal with the data.
    
    return (data_list)

Now lets check if my function worked.

In [13]:
temp = scrape_pdf('state_pdfs/al-alpha.pdf')
len(temp)

10043

In [14]:
10043/7

1434.7142857142858

There seems to be an issue. Had this worked properly, the above number should 1208 or at least a whole number. After reviewing the PDF, there is a newline character after some of the names. The following entry in the data is the `service` type. 

Since there is a limited number of military services, I will loop through the data checking the preceeding entires after the `name` column. If a name is broken into two lines, I will merge them into one. As long as the first entry is correct, this process will work for all the PDFs.

In [15]:
military = ['ARMY', 'NAVY', 'MARINE CORPS', 'AIR FORCE']

In [16]:
check = temp[:106]

counter = len(check)

while counter > 0:
    if check[1] not in military:
        
        print(check[1] + ' WHAT, WHAT?!')
        
        check[0] = check[0] + check[1]
        check.remove(check[1])
    
    print(check[:8])
    del check[:8]
    counter = len(check)

['ABRAMS TIMOTHY C JR', 'ARMY', 'PFC', '19461104', 'TUSCALOOSA', 'TUSCALOOSA', '19670103', 'Y']
['ABSTON JAMES ESTUS JR', 'ARMY', 'PFC', '19441102', 'COTTONDALE', 'TUSCALOOSA', '19670227', 'Y']
FRANKLIN WHAT, WHAT?!
['ACTON MARION FRANKLIN', 'ARMY', 'PFC', '19470901', 'HUNTSVILLE', 'MADISON', '19660411', 'Y']
['ADAMS JAMES CONRAD', 'ARMY', 'MAJ', '19381025', 'PHENIX CITY', 'RUSSELL', '19700501', 'Y']
['ADAMS SPENCER', 'ARMY', 'SSG', '19340506', 'MOBILE', 'MOBILE', '19680215', 'Y']
['ADAMS WALTER LEE', 'ARMY', 'PFC', '19461227', 'MCINTOSH', 'WASHINGTON', '19670610', 'Y']
['ADAMSON LARRY ONEAL', 'ARMY', 'SP4', '19460910', 'FAIRFAX', 'CHAMBERS', '19691003', 'Y']
['ADDISON ONEAL', 'ARMY', 'SP5', '19270506', 'WEST ENSLEY', 'JEFFERSON', '19660504', 'Y']
['ADKISON CARL ELMUS', 'ARMY', 'SP4', '19490531', 'GADSDEN', 'ETOWAH', '19690812', 'Y']
['AKINS ADRIAN ALAN', 'ARMY', 'SP5', '19490404', 'CORDOVA', 'WALKER', '19691025', 'Y']
['ALEXANDER BOBBY RAY', 'MARINE CORPS', 'CPL', '19470213', 'DECATUR

Fantastic, it works! now lets tweak this a bit and make a list of lists then I can throw into a `DataFrame` easily.

In [17]:
def prep_for_df(data_in_list, state):  #Takes in a list and the state, as a string. It will return a list of lists.
    
    big_long_list = data_in_list.copy()   #Making a deep copy, so I don't effect the underlying list. 
    output = []
    counter = len(big_long_list)

    while counter > 0:
        if big_long_list[1] not in military:

            big_long_list[0] = big_long_list[0] + ' ' + big_long_list[1]
            big_long_list.remove(big_long_list[1])

        place_holder = big_long_list[:8]
        place_holder.append(state)
        output.append(place_holder)
        del big_long_list[:8]
        counter = len(big_long_list)
    
    return(output)

In [18]:
bama = scrape_pdf('state_pdfs/al-alpha.pdf')
how_long = prep_for_df(bama, "AL")
len(how_long)

1208

In [19]:
blerg = pd.DataFrame(how_long, columns = ['name', 'service', 'rank_rate', 'birthdate', 'home_city', 'home_county', 'incident_death', 'body_rec', 'state'])

In [20]:
print(blerg.shape)
blerg.head()

(1208, 9)


,name,service,rank_rate,birthdate,home_city,home_county,incident_death,body_rec,state
0,ABRAMS TIMOTHY C JR,ARMY,PFC,19461104,TUSCALOOSA,TUSCALOOSA,19670103,Y,AL
1,ABSTON JAMES ESTUS JR,ARMY,PFC,19441102,COTTONDALE,TUSCALOOSA,19670227,Y,AL
2,ACTON MARION FRANKLIN,ARMY,PFC,19470901,HUNTSVILLE,MADISON,19660411,Y,AL
3,ADAMS JAMES CONRAD,ARMY,MAJ,19381025,PHENIX CITY,RUSSELL,19700501,Y,AL
4,ADAMS SPENCER,ARMY,SSG,19340506,MOBILE,MOBILE,19680215,Y,AL


Well it looks like I got everything all the peieces together. All that is is left to run through the folder of the PDFs, get the `state` from the file name, and let it go.

In [21]:
pdf_paths = []
depth = 1
#Getting the directory of the PDF files.
for dirpath, dirname, filenames in os.walk('state_pdfs'):
    for name in filenames:
        #The len(dirpath) is just to focus on the .pdf files without grabbing the other folders.
        if name.endswith('.pdf') == True and len(dirpath) == 10:
            pdf_paths.append(os.path.join(dirpath, name))
pdf_paths

['state_pdfs/me-alpha.pdf',
 'state_pdfs/al-alpha.pdf',
 'state_pdfs/la-alpha.pdf',
 'state_pdfs/nm-alpha.pdf',
 'state_pdfs/ks-alpha.pdf',
 'state_pdfs/il-alpha.pdf',
 'state_pdfs/sc-alpha.pdf',
 'state_pdfs/wy-alpha.pdf',
 'state_pdfs/mn-alpha.pdf',
 'state_pdfs/wa-alpha.pdf',
 'state_pdfs/ia-alpha.pdf',
 'state_pdfs/ar-alpha.pdf',
 'state_pdfs/tn-alpha.pdf',
 'state_pdfs/sd-alpha.pdf',
 'state_pdfs/pa-alpha.pdf',
 'state_pdfs/md-alpha.pdf',
 'state_pdfs/ca-alpha.pdf',
 'state_pdfs/oh-alpha.pdf',
 'state_pdfs/ct-alpha.pdf',
 'state_pdfs/mi-alpha.pdf',
 'state_pdfs/ny-alpha.pdf',
 'state_pdfs/ak-alpha.pdf',
 'state_pdfs/ky-alpha.pdf',
 'state_pdfs/hi-alpha.pdf',
 'state_pdfs/mo-alpha.pdf',
 'state_pdfs/fl-alpha.pdf',
 'state_pdfs/ut-alpha.pdf',
 'state_pdfs/nj-alpha.pdf',
 'state_pdfs/tx-alpha.pdf',
 'state_pdfs/de-alpha.pdf',
 'state_pdfs/co-alpha.pdf',
 'state_pdfs/va-alpha.pdf',
 'state_pdfs/vt-alpha.pdf',
 'state_pdfs/ok-alpha.pdf',
 'state_pdfs/in-alpha.pdf',
 'state_pdfs/mt-alph

Now lets throw it all together to test. I am going to verify it works for all the states and DC.

In [23]:
for pdf in pdf_paths:
    
    state = str(pdf[11:13].upper())
    
    print(pdf)
    print(state)
    
    place_holder = scrape_pdf(pdf)
    
    #print(place_holder)
    
    data = prep_for_df(place_holder, state)
    
    pd.DataFrame(data, columns = ['name', 'service', 'rank_rate', 'birthdate', 'home_city', 'home_county', 'incident_death', 'body_rec', 'state'])

state_pdfs/me-alpha.pdf
ME
state_pdfs/al-alpha.pdf
AL
state_pdfs/la-alpha.pdf
LA
state_pdfs/nm-alpha.pdf
NM
state_pdfs/ks-alpha.pdf
KS


IndexError: list index out of range

There seems to be an issue with the KS records, so going to investigate that.

In [24]:
ks_data = scrape_pdf('state_pdfs/ks-alpha.pdf')

In [25]:
print(len(ks_data))
ks_data[:34]

5262


['ACHESON CHARLES ',
 'RALPH',
 'ARMY',
 'PFC',
 '19470120',
 'PALCO',
 'ROOKS',
 '19671121',
 'Y',
 'ACOSTA JOSE FRANCISCO',
 'MARINE CORPS',
 'CPL',
 '19470525',
 'WICHITA',
 'SEDGWICK',
 '19671001',
 'Y',
 'ADAM JOHN QUINCY',
 'AIR FORCE',
 'SMSGT',
 '19471222',
 'BETHEL',
 'WYANDOTTE',
 '19680522',
 'N',
 'ADAMS THOMAS ',
 'EDWARD',
 'NAVY',
 'SN',
 '19451105',
 'TOPEKA',
 'SHAWNEE',
 '19690227',
 'Y']

In [26]:
prep_for_df(ks_data, 'KS')

IndexError: list index out of range

My prep function is not working as intended. I am going to make a trouble shooting one that I will add print statesments enable to troubleshoot.

In [28]:
def TS_prep_for_df(data_in_list, state):  #Takes in a list and the state, as a string. It will return a list of lists.
    
    big_long_list = data_in_list.copy()    #Making a deep copy, so I don't effect the underlying list. 
    
    output = []
    
    counter = len(big_long_list)

    while counter > 0:
        
        if big_long_list[1] not in military:

            big_long_list[0] = big_long_list[0] + ' ' + big_long_list[1]
            big_long_list.remove(big_long_list[1])

        place_holder = big_long_list[:8]
        place_holder.append(state)
        
        output.append(place_holder)
        
        print(place_holder)
        
        del big_long_list[:8]
        counter = len(big_long_list)
    
    #return(output)

In [29]:
TS_prep_for_df(ks_data, 'KS')

['ACHESON CHARLES  RALPH', 'ARMY', 'PFC', '19470120', 'PALCO', 'ROOKS', '19671121', 'Y', 'KS']
['ACOSTA JOSE FRANCISCO', 'MARINE CORPS', 'CPL', '19470525', 'WICHITA', 'SEDGWICK', '19671001', 'Y', 'KS']
['ADAM JOHN QUINCY', 'AIR FORCE', 'SMSGT', '19471222', 'BETHEL', 'WYANDOTTE', '19680522', 'N', 'KS']
['ADAMS THOMAS  EDWARD', 'NAVY', 'SN', '19451105', 'TOPEKA', 'SHAWNEE', '19690227', 'Y', 'KS']
['ADAMS WILLIAM CARL', 'MARINE CORPS', 'SGT', '19451002', 'JUNCTION CITY', 'GEARY', '19690823', 'Y', 'KS']
['ALDERMAN ANDREW  ALBERT', 'NAVY', 'PO3', '19440512', 'COFFEYVILLE', 'MONTGOMERY', '19670317', 'Y', 'KS']
['ALLENDORF MICHAEL  GEORGE', 'ARMY', 'CPL', '19480702', 'TOPEKA', 'SHAWNEE', '19690311', 'Y', 'KS']
['ALLGOOD FRANKIE  EUGENE', 'MARINE CORPS', 'LTCOL', '19300501', 'FT SCOTT', 'BOURBON', '19680326', 'N', 'KS']
['ALVORD RONNIE EUGENE', 'ARMY', 'PFC', '19470111', 'HUTCHINSON', 'RENO', '19680305', 'Y', 'KS']
['AMERINE KENT L', 'ARMY', 'SP4', '19430404', 'GREAT BEND', 'BARTON', '19660802

IndexError: list index out of range

Well I found the error. It looks like if the county or city name was too long, they also created a newline for it. It efforts to account for all this, I will alter my `scrape_pdf` function.

After some delibration, I couldn't find a way to tell the difference between the records that contained a newline on the `county` record or on the `home` record. These records will have to be seperately dealt with. Since county data can be more easily grouped, I will condense on the `home` records in hopes that the county names are all one liners. I will gather lists of counties in all states to verify that data when I proceede later on.

In [30]:
def TS_prep_for_df(data_in_list, state):  #Takes in a list and the state, as a string. It will return a list of lists.
    
    big_long_list = data_in_list.copy()    #Making a deep copy, so I don't effect the underlying list. 
    output = []
    counter = len(big_long_list)

    while counter > 0:
        
        #This loop checks if there is newline in the name line
        
        if big_long_list[1] not in military:

            big_long_list[0] = big_long_list[0] + big_long_list[1]
            
            big_long_list.remove(big_long_list[1])
            
        #New code to account for the new lines in the home and county entries 
        #This checks the list to see if any new lines are present
        try:
            int(big_long_list[6])
            
        except ValueError:
            
            #If there are two newlines, one in home and one in county, this will be taken care of here
            try:
                int(big_long_list[7])
                
            except ValueError:
                big_long_list[5] = big_long_list[5] + big_long_list[6]
                del big_long_list[6]
                
            #If there is only one newline, this will put the middle entry into the home cell
            try:
                int(big_long_list[6])
                
            except ValueError:
                big_long_list[4] = big_long_list[4] + big_long_list[5]
                del big_long_list[5]
        
        place_holder = big_long_list[:8]
        
        
        place_holder.append(state)
        
        output.append(place_holder)
        
        del big_long_list[:8]
        counter = len(big_long_list)
    
    return(output)

In [31]:
to_inspect = TS_prep_for_df(ks_data, 'KS')
to_inspect[-2]

['ZELLER MICHAEL CHARLES',
 'ARMY',
 'SP4',
 '19470902',
 'WAMEGOPOTTAWATOMI',
 'E',
 '19680225',
 'Y',
 'KS']

So it worked, mostly. The above is an example of where my solution fails. The next iteration of this function should reference the counties, per state, and verify if the name is correct. I am going to move forward at this point. I am going to formalize the `prep_for_df` function and see if I can put it altogether.

In [32]:
def prep_for_df(data_in_list, state):  #Takes in a list and the state, as a string. It will return a list of lists.
    
    big_long_list = data_in_list.copy()    #Making a deep copy, so I don't effect the underlying list. 
    output = []
    counter = len(big_long_list)

    while counter > 0:
        
        #This loop checks if there is newline in the name line
        
        if big_long_list[1] not in military:

            big_long_list[0] = big_long_list[0] + big_long_list[1]
            
            big_long_list.remove(big_long_list[1])
            
        #New code to account for the new lines in the home and county entries 
        #This checks the list to see if any new lines are present
        try:
            int(big_long_list[6])
            
        except ValueError:
            
            #If there are two newlines, one in home and one in county, this will be taken care of here
            try:
                int(big_long_list[7])
                
            except ValueError:
                big_long_list[5] = big_long_list[5] + big_long_list[6]
                del big_long_list[6]
                
            #If there is only one newline, this will put the middle entry into the home cell
            try:
                int(big_long_list[6])
                
            except ValueError:
                big_long_list[4] = big_long_list[4] + big_long_list[5]
                del big_long_list[5]
        
        place_holder = big_long_list[:8]
        
        
        place_holder.append(state)
        
        output.append(place_holder)
        
        del big_long_list[:8]
        counter = len(big_long_list)
    
    return(output)

Another trial of it all together.

In [33]:
df_list = []

for pdf in pdf_paths:
    
    state = str(pdf[11:13].upper())
    
    place_holder = scrape_pdf(pdf)
    
    data = prep_for_df(place_holder, state)
    
    df_list.append(pd.DataFrame(data, columns = ['name', 'service', 'rank_rate', 'birthdate', 'home_city', 'home_county', 'incident_death', 'body_rec', 'state']))

In [34]:
len(df_list)

51

Now combine them all

In [35]:
state_records = pd.concat(df_list)

In [36]:
state_records.shape

(57646, 9)

In [37]:
state_records.state.value_counts()

CA    5572
NY    4118
TX    3415
PA    3147
OH    3093
IL    2935
MI    2656
FL    1954
NC    1612
GA    1581
IN    1534
NJ    1487
MO    1417
MA    1331
VA    1304
TN    1295
AL    1208
WI    1160
MN    1077
KY    1055
WA    1047
MD    1013
OK     986
SC     895
LA     885
IA     851
WV     733
OR     710
MS     636
KS     627
CO     623
AZ     619
CT     612
AR     591
NE     396
NM     395
UT     361
ME     341
HI     276
MT     267
DC     242
NH     226
ID     217
RI     209
ND     199
SD     192
NV     149
DE     122
WY     118
VT     100
AK      57
Name: state, dtype: int64

In [19]:
#state_records.to_pickle('states_df')
state_records = pd.read_pickle('states_df')

In [20]:
state_records.head()

,name,service,rank_rate,birthdate,home_city,home_county,incident_death,body_rec,state
0,ABDELLAH BRUCE ALLYN,ARMY,PVT,19521215,EAST HOLDEN,PENOBSCOT,19711026,Y,ME
1,ALBERT LOUIS BASIL JR,MARINE CORPS,PFC,19480928,DANFORTH,WASHINGTON,19691008,Y,ME
2,ALBERT RICHARD PATRICK,ARMY,CPL,19480317,ST AGATHA,AROOSTOOK,19690820,Y,ME
3,ALEXANDER DAVID HAROLD,ARMY,SP4,19480118,SANFORD,YORK,19681216,Y,ME
4,ALLEN KEITH DOBSON JR,ARMY,CW2,19461209,WESTFIELD,AROOSTOOK,19690413,Y,ME


## Additional Database

Another document take from the National Archives.

In [74]:
pdfFileObj = open('DCAS.VN08.DOC.pdf', 'rb')

pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

pdf_data = ''

for pg in list(range(2)):
    pageObj = pdfReader.getPage(pg)
    chunk = pageObj.extractText()
    pdf_data += chunk

In [75]:
pdf_data

"Vietnam Conflict and Korean War File Layout (Pipe (|) Delimited Format)#Column NamesDataTypeDataSizeCoulmn DescriptionFurther Column Description1SERVICE NOText10Service Number\n2CText1Member Component Code\n\n3PTPText1Person Type Name Code\n\n4Person Type NameText55Person Type Name\n\n5Member NameText40Member Name\n6SText1Member Service Code\nA= Army, F = Air Force,M = Marines Corps, N = Navy7Service NameText50Member Service Name\n8Rank RateText6Member Rank or Rate\n\n9PGText3Member Paygrade\n10OccText6Member Occupation Code\n\n11Occupation NameText250Member Occupation Name\n\n12Birth DtText8Member BirthdateYYYYMMDD\n\n13GText1Member's GenderM = Male, F= Female\n\n14Hor CityText50Home of Record City\n\n15Hor CountyText50Home of Record County\n\n16Hor CtryText2Home of Record Country Code\n\n17Hor STText2Home of Record State Code\n\n18State Prv NmText50State or Province Name\n\n19Marital StatusText50Marital Name\n\n20Religion NameText60Religion Short Name\n\n21LText2Religion Code\n\n22R

In [24]:
pdf_data.splitlines()

['Vietnam Conflict and Korean War File Layout (Pipe (|) Delimited Format)#Column NamesDataTypeDataSizeCoulmn DescriptionFurther Column Description1SERVICE NOText10Service Number',
 '2CText1Member Component Code',
 '',
 '3PTPText1Person Type Name Code',
 '',
 '4Person Type NameText55Person Type Name',
 '',
 '5Member NameText40Member Name',
 '6SText1Member Service Code',
 'A= Army, F = Air Force,M = Marines Corps, N = Navy7Service NameText50Member Service Name',
 '8Rank RateText6Member Rank or Rate',
 '',
 '9PGText3Member Paygrade',
 '10OccText6Member Occupation Code',
 '',
 '11Occupation NameText250Member Occupation Name',
 '',
 '12Birth DtText8Member BirthdateYYYYMMDD',
 '',
 "13GText1Member's GenderM = Male, F= Female",
 '',
 '14Hor CityText50Home of Record City',
 '',
 '15Hor CountyText50Home of Record County',
 '',
 '16Hor CtryText2Home of Record Country Code',
 '',
 '17Hor STText2Home of Record State Code',
 '',
 '18State Prv NmText50State or Province Name',
 '',
 '19Marital Status

In [97]:
to_clean = pdf_data.splitlines()
col_desc = []
for line in to_clean:
    col = re.search(".*(?=Text)", line)
    description = re.search("(?<=Text).*", line)
    try:
        col_data = col.group(0)
        desc_data = description.group(0)
        
        place_holder = col_data, desc_data
        col_desc.append(place_holder)
    except AttributeError:
        pass

In [98]:
col_desc[0] = col_desc[0][0][145:], col_desc[0][1]
col_desc[6] = col_desc[6][0][51:], col_desc[6][1]
col_desc[35] = col_desc[35][0][145:], col_desc[35][1]
col_desc[37] = col_desc[37][0][36:], col_desc[37][1]
col_desc[43] = col_desc[43][0][36:], col_desc[43][1]

In [99]:
def remove_num(list):
    
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list

In [102]:
col_names = []
for line in col_desc:
    col_names.append(remove_num(line))

In [103]:
col_names

[['SERVICE NO', 'Service Number'],
 ['C', 'Member Component Code'],
 ['PTP', 'Person Type Name Code'],
 ['Person Type Name', 'Person Type Name'],
 ['Member Name', 'Member Name'],
 ['S', 'Member Service Code'],
 ['Service Name', 'Member Service Name'],
 ['Rank Rate', 'Member Rank or Rate'],
 ['PG', 'Member Paygrade'],
 ['Occ', 'Member Occupation Code'],
 ['Occupation Name', 'Member Occupation Name'],
 ['Birth Dt', 'Member BirthdateYYYYMMDD'],
 ['G', "Member's GenderM = Male, F= Female"],
 ['Hor City', 'Home of Record City'],
 ['Hor County', 'Home of Record County'],
 ['Hor Ctry', 'Home of Record Country Code'],
 ['Hor ST', 'Home of Record State Code'],
 ['State Prv Nm', 'State or Province Name'],
 ['Marital Status', 'Marital Name'],
 ['Religion Name', 'Religion Short Name'],
 ['L', 'Religion Code'],
 ['Race Name', 'Race Name'],
 ['Ethnic Name', 'Ethnic Short Name'],
 ['Race Omb', 'Race OMB Name'],
 ['Ethnic Group Name', 'Ethnic Group Name'],
 ['Cas Circumstances', 'Casualty Circumstance

In [104]:
columns = []
for description in col_names:
    columns.append(description[0])

In [127]:
#columns = 
df_2 = pd.read_csv('DCAS.VN.EXT08.DAT', delimiter = '|', engine='python', header=None, names=columns)
df_2.head()

,SERVICE NO,C,PTP,Person Type Name,Member Name,S,Service Name,Rank Rate,PG,Occ,Occupation Name,Birth Dt,G,Hor City,Hor County,Hor Ctry,Hor ST,State Prv Nm,Marital Status,Religion Name,L,Race Name,Ethnic Name,Race Omb,Ethnic Group Name,Cas Circumstances,Cas City,Cas ST,Cas Ctry,Cas Region Name,Country Or Water Name,Unit Name,D,Process Dt,Death Dt,Year,WC,OITP,OI Name,OI Location,Close Dt,Aircraft,H,Casualty Type Name,Casualty Category,Casualty Reason Name,CSN,Body,RecoveredCasualty Closure Name,Wall,Incident Category,I Status Dt,I CSN,I H,I Aircraft
0,503503584,Y,S,SELECTED SERVICE,AADLAND GERALD L,A,ARMY,SP4,E04,11B,INFANTRYMAN,19450329,M,SISSETON,ROBERTS,US,SD,SOUTH DAKOTA,NEVER MARRIED,LUTHERAN CHURCHES,LL,WHITE,NOT SPECIFIED,WHITE,UNKNOWN,NaN,KONTUM,M2,VS,ASIA,SOUTH VIETNAM,D CO 3 BN 12 INF 4 INF DIV,Z,19680501,19680530,1968,V,C,VIETNAM CONFLICT,SOUTHEAST ASIA,19680530.0,NaN,H,DECEASED,KILLED IN ACTION,SMALL ARMS FIRE,NaN,Y,BURIED - UNKNOWN DISP,63W 014,NaN,NaN,NaN,NaN,NaN
1,465720672,Y,S,SELECTED SERVICE,AALUND JAMES DOWNING,A,ARMY,SGT,E05,11B,INFANTRYMAN,19450511,M,HOUSTON,HARRIS,US,TX,TEXAS,NEVER MARRIED,METHODIST CHURCHES,MM,WHITE,NOT SPECIFIED,WHITE,UNKNOWN,NaN,PHUOC LONG,M3,VS,ASIA,SOUTH VIETNAM,1 CAV DIV AIRMOBILE,Z,19700301,19700228,1970,V,C,VIETNAM CONFLICT,SOUTHEAST ASIA,19700228.0,NaN,H,DECEASED,KILLED IN ACTION,ARTILLERY/MORTAR/ROCKET,NaN,Y,BURIED - UNKNOWN DISP,13W 066,NaN,NaN,NaN,NaN,NaN
2,473635900,R,K,ACTIVE - REGULAR,AAMOLD DANIEL LAWRENCE,F,AIR FORCE,A1C,E03,,NaN,19510823,M,MOORHEAD,CLAY,US,MN,MINNESOTA,NEVER MARRIED,BAPTIST CHURCHES,BB,WHITE,NOT SPECIFIED,WHITE,UNKNOWN,NaN,NINH THUAN,M2,VS,ASIA,SOUTH VIETNAM,NaN,Z,19700801,19700820,1970,V,C,VIETNAM CONFLICT,SOUTHEAST ASIA,19700820.0,NaN,NH,DECEASED,ACCIDENT,OTHER ACCIDENT,NaN,Y,BURIED - UNKNOWN DISP,08W 124,NaN,NaN,NaN,NaN,NaN
3,2221574,Y,S,SELECTED SERVICE,AARDE JAMES RAYMOND,M,MARINE CORPS,LCPL,E03,0311,RIFLEMAN,19440608,M,KENT,KING,US,WA,WASHINGTON,MARRIED,"PROTESTANT, NO DENOMINATIONAL PREFERENCE",TN,WHITE,NOT SPECIFIED,WHITE,UNKNOWN,NaN,QUANG TRI,M1,VS,ASIA,SOUTH VIETNAM,NaN,Z,19661201,19661229,1966,V,C,VIETNAM CONFLICT,SOUTHEAST ASIA,19661229.0,NaN,H,DECEASED,DIED OF WOUNDS,EXPLOSIVE DEVICE,NaN,Y,BURIED - UNKNOWN DISP,13E 095,WOUNDED IN ACTION (VERY SERIOUS),0.0,WIA,H,NaN
4,025368444,Y,S,SELECTED SERVICE,AARON CHARLES EDWARD,A,ARMY,SP4,E04,11B,INFANTRYMAN,19481212,M,FORGE VILLAGE,MIDDLESEX,US,MA,MASSACHUSETTS,NEVER MARRIED,METHODIST CHURCHES,MM,WHITE,NOT SPECIFIED,WHITE,UNKNOWN,NaN,BINH THUAN,M2,VS,ASIA,SOUTH VIETNAM,1 FIELD FORCE,Z,19700501,19700505,1970,V,C,VIETNAM CONFLICT,SOUTHEAST ASIA,19700505.0,NaN,H,DECEASED,KILLED IN ACTION,GRENADE,NaN,Y,BURIED - UNKNOWN DISP,11W 098,NaN,NaN,NaN,NaN,NaN


In [125]:
df_2.isna().sum()

SERVICE NO                            0
C                                   182
PTP                                   0
Person Type Name                      3
Member Name                           0
S                                     0
Service Name                          0
Rank Rate                             0
PG                                    0
Occ                                  28
Occupation Name                    4207
Birth Dt                              0
G                                     0
Hor City                            122
Hor County                           29
Hor Ctry                              0
Hor ST                              117
State Prv Nm                        117
Marital Status                        0
Religion Name                         3
L                                     3
Race Name                             0
Ethnic Name                           0
Race Omb                              0
Ethnic Group Name                     0


In [130]:
df_2['Cas City'].value_counts()

QUANG NAM          8090
QUANG TRI          7540
THUA THIEN         4289
QUANG TIN          2987
QUANG NGAI         2981
BINH DUONG         2766
TAY NINH           2683
BINH DINH          2418
KONTUM             1691
HUA NGHIA          1442
BIEN HOA           1334
GIA DINH           1318
PLEIKU             1092
LONG AN            1020
BINH LONG           947
DINH TUONG          835
PHUOC LONG          695
LONG KHANH          577
KIEN HOA            462
KHANH HOA           449
OFFSHORE            329
PHU YEN             326
BINH THUAN          302
PHUOC TUY           238
PHONG DINH          198
BINH THUY           195
DARLAC              192
NINH THUAN          189
QUANG DUC           180
VINH LONG           169
LAM DONG            167
KIEN TUONG          158
UNKNOWN             136
KIEN GIANG          124
CHAU DOC             84
BA XUGEN             82
TUYEN DUC            80
KIEN PHONG           79
VINH BINH            71
GO CONG              71
AN XUGEN             70
CHUONG THIEN    

In [109]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58220 entries, 0 to 58219
Data columns (total 55 columns):
SERVICE NO                        58220 non-null object
C                                 58038 non-null object
PTP                               58220 non-null object
Person Type Name                  58217 non-null object
Member Name                       58220 non-null object
S                                 58220 non-null object
Service Name                      58220 non-null object
Rank Rate                         58220 non-null object
PG                                58220 non-null object
Occ                               58192 non-null object
Occupation Name                   54013 non-null object
Birth Dt                          58220 non-null int64
G                                 58220 non-null object
Hor City                          58098 non-null object
Hor County                        58191 non-null object
Hor Ctry                          58220 non-null obj

In [124]:
for col in df_2.columns:
    print(f"These are the all the values in {col}")
    print(df_2[col].unique())
    print(f"TOTAL {len(df_2[col].unique())}")
    print('------------------------------------------')

These are the all the values in SERVICE NO
['503503584' '465720672' '473635900' ... '56957866' '16829253' '2351252']
TOTAL 58220
------------------------------------------
These are the all the values in C
['Y' 'R' 'V' nan 'G']
TOTAL 5
------------------------------------------
These are the all the values in PTP
['S' 'K' 'L' 'T' 'Q' 'R' 'F']
TOTAL 7
------------------------------------------
These are the all the values in Person Type Name
['SELECTED SERVICE' 'ACTIVE - REGULAR' 'ACTIVE - GUARD/RESERVE'
 'TEMP/PERM DISABILITY RETIRED LIST (OVER 120 DAYS)'
 'TEMP/PERM DISABILITY RETIRED LIST (120 DAYS OR LESS)' nan
 'RETIRED/SEPARATED 120 DAYS OR LESS (NOT T/PDRL)']
TOTAL 7
------------------------------------------
These are the all the values in Member Name
['AADLAND GERALD L' 'AALUND JAMES DOWNING' 'AAMOLD DANIEL LAWRENCE' ...
 'ZYPH JAMES LOUIS' 'ZYWICA GARY ROMAN' 'ZYWICKE DAVID LEE']
TOTAL 57933
------------------------------------------
These are the all the values in S
['A' 'F' 

https://www.archives.gov/research/military/vietnam-war/casualty-lists/state-level-alpha.html for state by state

https://catalog.archives.gov/id/2240992 for the `.dat` file. Contains a lot of the big stuff.

https://amaral.northwestern.edu/blog/step-step-how-plot-map-slider-represent-time-evolu for the cholorplot slider

https://catalog.archives.gov/id/10643608 list and just year of operation
http://id.loc.gov/authorities/subjects/sh85143280.html 

## Link1

## Link2

## Link3

## Link4

## Link5 

Testing

In [ ]:
huh = ['asf', 'asd','pqiwr', '452', 'fuck ', ' off', 'no', 'way!', '239', 'lsdf']
what = ['asf', 'asd','pqiwr', '452', 'will', 'this', 'work', '918364', 'kjsfd']
where = ['asf', 'asd','pqiwr', '452', 'NO', 'MOVEMENT', '918364', 'uhas']

In [ ]:
batch = [huh,what,where]

for trial in batch:
    
    try: 
        int(trial[6])
    
    except ValueError:
        try:
            int(trial[7])
        except ValueError:
            print('this is the error' + ' ' + trial[7])
            trial[5] = trial[5] + trial[6]
            del trial[6]
    
        try:
            int(trial[6])
        except ValueError:
            print('this is the error' + ' ' + trial[6])
            trial[4] = trial[4] + trial[5]
            del trial[5]

batch